In [1]:
# -*- coding: utf-8 -*-
"""
Kịch bản huấn luyện cuối cùng, sử dụng bộ dữ liệu đã được đơn giản hóa
và cặp đôi Optuna + LightGBM.
"""

import pandas as pd
import numpy as np
import os
import lightgbm as lgb
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score # Dùng accuracy để có thể so sánh trực tiếp với điểm Kaggle

optuna.logging.set_verbosity(optuna.logging.WARNING)

# Tải dữ liệu
print("--- Tải dữ liệu từ Data_clean_v4 ---")
try:
    save_dir = "Data_clean_v4"
    train_df = pd.read_csv(f'{save_dir}/train_cleaned.csv')
    test_df = pd.read_csv(f'{save_dir}/test_cleaned.csv')
    X = train_df.drop(columns=['Survived', 'PassengerId'])
    y = train_df['Survived']
    X_test = test_df.drop(columns=['PassengerId'])[X.columns]
except Exception as e:
    print(f"Lỗi: {e}. Hãy chạy file pipeline_final_simplified.py trước.")
    exit()

# Tối ưu hóa với Optuna
print("--- Tối ưu hóa siêu tham số ---")
def objective(trial):
    param = {
        'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'boosting_type': 'gbdt',
        'n_estimators': 1000, # Dùng n_estimators lớn và early stopping
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 10, 40),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    scores = []
    for train_idx, val_idx in cv.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        model = lgb.LGBMClassifier(**param)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[lgb.early_stopping(50, verbose=False)])
        preds = model.predict(X_val)
        scores.append(accuracy_score(y_val, preds))
    return np.mean(scores)

study = optuna.create_study(direction='maximize', study_name='Final LGBM Optimization')
study.optimize(objective, n_trials=100) # Tăng số lần thử lên 100
best_params = study.best_params

print(f"Điểm Accuracy tốt nhất (cross-validated): {study.best_value:.5f}")
print("Các siêu tham số tốt nhất:")
print(best_params)

# Huấn luyện lại và tạo submission
print("--- Huấn luyện lại và tạo file submission ---")
final_model = lgb.LGBMClassifier(objective='binary', verbosity=-1, **best_params)
final_model.fit(X, y)
predictions = final_model.predict(X_test)

submission_df = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': predictions})
# --- Lưu trữ theo cấu trúc của bạn ---
save_dir = "Pred_Result"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    print(f"Đã tạo thư mục '{save_dir}'")

try:

    # 2. Lưu các file CSV đã được làm sạch hoàn chỉnh
    submission_df.to_csv(f'{save_dir}/pred_result_Lab_1_v7.csv', index=False)
    print(f"- Đã lưu pred_result_Lab_1_v7 vào '{save_dir}'")

    print("\nNội dung thư mục 'Pred_Result':")
    print(os.listdir(save_dir))

except Exception as e:
    print(f"\nLỗi khi lưu file: {e}")

print("\nHoàn tất! File 'final_submission.csv' đã được tạo.")

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- Tải dữ liệu từ Data_clean_v4 ---
--- Tối ưu hóa siêu tham số ---
Điểm Accuracy tốt nhất (cross-validated): 0.84733
Các siêu tham số tốt nhất:
{'learning_rate': 0.04778601881136146, 'num_leaves': 37, 'max_depth': 8, 'lambda_l1': 0.01184213921929606, 'lambda_l2': 0.003408649447684883, 'feature_fraction': 0.9905961887384869, 'bagging_fraction': 0.8781783791206644, 'bagging_freq': 7}
--- Huấn luyện lại và tạo file submission ---
- Đã lưu pred_result_Lab_1_v7 vào 'Pred_Result'

Nội dung thư mục 'Pred_Result':
['pred_result_Lab_1.csv', 'pred_result_Lab_1_v2.csv', 'pred_result_Lab_1_v3.csv', 'pred_result_Lab_1_v4.csv', 'pred_result_Lab_1_v5.csv', 'pred_result_Lab_1_v6.csv', 'pred_result_Lab_1_v7.csv']

Hoàn tất! File 'final_submission.csv' đã được tạo.
